In [12]:
# -*- coding: utf-8 -*-
"""MongoDB_test.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1Wrfo5GA8M5Q_DY3_SZ5Y0wdll-mZvj8d

**連線資料庫**
"""

# coding=UTF-8

from pymongo import MongoClient
from datetime import datetime
import time
import psutil
#import urllib.parse

'''連結mongodb不須認證'''
def connect2MongoDB(url='mongodb://127.0.0.1'):
    client = MongoClient(url)
    return client

def getDatabase(client, databaseName='test'):
    return client[databaseName]

def getCollectionCursor(db, collectionName='test1'):
    return db[collectionName]

#計算時間
def timeSpent(f):
    def wrapper(*args, **kwargs):
        start=datetime.now()
        response = f(*args, **kwargs)
        print(f.__name__)
        print ('Time:', datetime.now()-start)
        return response
    return wrapper

"""**資料處理**"""

#資料處理

from obspy import UTCDateTime, Stream, Trace
import struct
import os
import numpy as np

def unpackAfile(infile):
    temp = []
# == opening Afile ==
    b= os.path.getsize(infile)
    FH = open(infile, 'rb')
    line = FH.read(b)
    fileHeader= struct.unpack("<4s3h6bh6s", line[0:24])
    
    fileLength = fileHeader[3]
    port = fileHeader[10]
    FirstStn = fileHeader[11][0:4].decode('ASCII').rstrip()
    #print(fileHeader)
    #print(FirstStn)
    #print(port)

# =================================Header===================================
    
    portHeader = []
    for i in range(24,port*32,32):
        port_data = struct.unpack("<4s4s3sbh2b4s12b",line[i:i+32])
        portHeader.append(port_data)
        
    
    #print(portHeader)
# =================================Data===================================    

    dataStartByte = 24+int(port)*32
    dataPoint = 3*int(port)*int(fileLength)*100
    times = int(port)*3*4
    data=[]

    data = struct.unpack("<%di"%dataPoint,line[dataStartByte:dataStartByte+dataPoint*4])

    
    portHeader = np.array(portHeader)    
    data = np.array(data)    
    idata =data.reshape((3,port,fileLength*100),order='F')
    
#== write to obspy Stream --
    #print(len(idata[0][0]))
    sttime = UTCDateTime(fileHeader[1],fileHeader[4],fileHeader[5],fileHeader[6],fileHeader[7],fileHeader[8],fileHeader[2])
    sttime = datetime(fileHeader[1],fileHeader[4],fileHeader[5],fileHeader[6],fileHeader[7],fileHeader[8],fileHeader[2])
    
    #print(fileHeader[1],fileHeader[4],fileHeader[5],fileHeader[6],fileHeader[7],fileHeader[8],fileHeader[2])
    npts = fileHeader[3]*fileHeader[9]
    samp = fileHeader[9]
    #print(sttime)
    # afst = Afile's Stream
    afst = Stream()
    for stc in range(fileHeader[10]):#port
        stn = portHeader[stc][0].decode('ASCII').rstrip()
        instrument = portHeader[stc][1].decode('ASCII').rstrip()
        loc = '0'+str(portHeader[stc][6].decode('ASCII'))
        net = str(portHeader[stc][7].decode('ASCII')).rstrip()
        GPS = int(portHeader[stc][3])
        # remove GPS unlock or broken station
        if ( GPS == 1 or GPS == 2 ):
            chc = 0
            if instrument == 'FBA':
                chc = 1
            elif instrument == 'SP':
                chc = 4
            elif instrument == 'BB':
                chc = 7
            
            #print(chc,instrument)
            
            # for each channel in port
            for ch in range(3):
                #print(num,ch,chc)
                chn = 'Ch'+str(chc+ch)
                #print(stc,channel)
                
                stats = {'network': net, 'station': stn, 'location': loc,
                        'channel': chn, 'npts': npts, 'sampling_rate': samp,
                        'starttime': sttime}
                
                data = np.array(idata[ch][stc], dtype=float)
                #print(data.shape)
                sttmp = Stream([Trace(data=data, header=stats)])
                afst += sttmp
                data = np.array2string(data, separator=' ')
                stats['data'] = data
                temp.append(stats)
    
    #stst = stms[0]
    #stst.detrend('linear').plot()
    #stms.write('test.mseed', format='MSEED')    
    #print(afst[34].stats)
    #print(afst[35].stats)
    #afst.write("example.mseed", format="MSEED")
    return afst, FirstStn, fileHeader, temp
      
afst, FirstStnm, fileHeader, data = unpackAfile('./05290646.40C')

"""測試範例說明:

  .40c 檔案有36筆資料
  分別已2萬、4萬、6萬、8萬、10萬 筆資料作效能測試
  
  
  新增資料部分 因為MongoDB 用 _id 當unique key
重複資料插入時，可能獲取的 _id相同而造成操作錯誤，
所以在代碼里手動指定 _id 避免錯誤

**測試範例一**

新增資料至資料庫集合 (Database collection)

JsonData 接受格式為python dict資料結構
"""

# 新增資料

def insert2MongoDB(collection, JsonData):
    collection.insert_many(JsonData)
    cpu=psutil.cpu_percent()
    memory=psutil.virtual_memory()
    memory=memory.available
    memory=memory/1024
    memory=memory/1024
    memory=memory/1024
    return cpu, memory

@timeSpent
def insertTest(times, data, database, collection):
    client = connect2MongoDB()
    db = getDatabase(client, database)
    collection = getCollectionCursor(db, collection)
    index = 1
    cpu_counter=[]
    memory_counter=[]
    #print(data)
    if isinstance(data, list):
        for i in range(times):
            for d in data:
                d['_id'] = index
                index += 1
            #print(data)
            temp1, temp2 = insert2MongoDB(collection, data)
            cpu_counter.append(temp1)
            memory_counter.append(temp2)
        #print(i)
        #print(times)        
    client.close()
    return cpu_counter, memory_counter

"""**測試範例二**

讀取collection內所有資料
"""

# 讀取collection內所有的資料

def selectAll(collection, limit=500):
    for data in collection.find().limit(limit):
        #print(data)
        pass
      
@timeSpent
def selectAllQuery(database, collection):
    start=time.time()
    client = connect2MongoDB()
    db = getDatabase(client, database)
    collection = getCollectionCursor(db, collection)                
    selectAll(collection)
    client.close()
    end = time.time()
    end = end-start
    end = round(end,4)
    return end

"""**測試範例三**

讀取collection內指定測站的資料
"""

# 讀取collection內特定測站的資料

def selectStation(collection, station = "TAP"):
    for data in collection.find({"station": station}):
        #print(data)
        pass

  
@timeSpent
def selectStationQuery(database, collection):
    client = connect2MongoDB()
    db = getDatabase(client, database)
    collection = getCollectionCursor(db, collection)                
    selectStation(collection)
    client.close()
    return

"""**測試範例四**

依時間區段從collection內讀取資料
"""

# 依時間區段讀取資料

def selectDateRange(collection, start = datetime(2018, 5, 24, 7, 51, 4), end = datetime(2018, 5, 24, 7, 52, 4)):
    for data in collection.find( {'starttime': {'$lt': end, '$gte': start}}):
        #print(data)
        pass

@timeSpent
def selectDateRangeQuery(database, collection):
    client = connect2MongoDB()
    db = getDatabase(client, database)
    collection = getCollectionCursor(db, collection)
    selectDateRange(collection)
    client.close()
    return

"""**測試範例五**

計算依時間區段從collection內讀取資料的資料筆數
"""

# 依時間區段讀取資料筆數

def selectDateRangeCount(collection, start = datetime(2018, 9, 24, 7, 51, 4), end = datetime(2018, 9, 24, 7, 52, 4)):
    count = collection.find( {'starttime': {'$lt': end, '$gte': start}}).count
    #print(count)
    
@timeSpent
def selectDateRangeCountQuery(database, collection):
    client = connect2MongoDB()
    db = getDatabase(client, database)
    collection = getCollectionCursor(db, collection)
    selectDateRangeCount(collection)
    client.close()
    return

"""**測試範例六**

匹配網路後依測站將資料分群統計
"""

# 匹配網路後依測站將資料分群統計

def pipelineAggregate(collection):
    pipeline = [{'$match': {'network': 'SMT'}},{'$group': {"_id": "$station", "count": {"$sum": 1}}}]
    result = collection.aggregate(pipeline)   
    #print(list(result))

@timeSpent        
def pipelineQuery(database, collection):
    client = connect2MongoDB()
    db = getDatabase(client, database)
    collection = getCollectionCursor(db, collection)
    pipelineAggregate(collection)
    client.close()
    return

def averagecpuusage(cpu_usage):
    total_cpu = sum(cpu_usage)
    total_cpu= total_cpu/len(cpu_usage)
    total_cpu=round(total_cpu)
    return total_cpu
def averagememoryusage(memory_usage):
    total_memory = sum(memory_usage)
    total_memory = total_memory/len(memory_usage)
    total_memory=round(total_memory,2)
    return total_memory

# 20W/36 = 5556 資料20萬筆 case 1
# 5W/36 = 1390 資料5萬筆 case 2
caseInfo = ['20萬筆資料','5萬筆資料']
caseInfo1 = ['200000','50000']
databaseCollection = [{'database':'test','collection':'test1', 'Loops':5556},
                      {'database':'test','collection':'test2', 'Loops':1390}
                     ]
usage_time=[]
cpu_usage_percent=[]
memmory_usage=[]
select_time=[]

for i in range(2):
  time_a = time.time()
  print('Case: ', i+1, ' ', caseInfo[i])
  cpu1, memory1 = insertTest(databaseCollection[i]['Loops'], data, databaseCollection[i]['database'], databaseCollection[i]['collection'])
  #print(cpu1)
  #print(memory1)
  time_temp=selectAllQuery(databaseCollection[i]['database'], databaseCollection[i]['collection'])
  select_time.append(time_temp)
  selectStationQuery( databaseCollection[i]['database'], databaseCollection[i]['collection'])
  selectDateRangeQuery( databaseCollection[i]['database'], databaseCollection[i]['collection'])
  selectDateRangeCountQuery( databaseCollection[i]['database'], databaseCollection[i]['collection'])
  pipelineQuery( databaseCollection[i]['database'], databaseCollection[i]['collection'])
  time_b = time.time()
  print("CPU used in %d seconds: %s Percent" % (time_b-time_a,averagecpuusage(cpu1)))
  print("Memory used in %d seconds: %s GB" % (time_b-time_a,averagememoryusage(memory1)))
  usage_time.append(round(time_b-time_a,2))
  cpu_usage_percent.append(averagecpuusage(cpu1))
  memmory_usage.append(averagememoryusage(memory1))
  


import csv
import codecs
Database = dict()
with open ('mongo_usage.csv', 'w', newline = '', encoding='big5') as csvfile:
    datanames=['CaseCount', 'Usage_time', 'CpuCount', 'Cpu_percent', 'Cpu_usage', 'Memory_usage', 'Select_AllData_time']
    writer = csv.DictWriter(csvfile , fieldnames= datanames)
    writer.writeheader()
    for i in range(0,len(usage_time)):
        Database['CaseCount']=caseInfo1[i]
        Database['Usage_time']=usage_time[i]
        Database['CpuCount']=psutil.cpu_count()
        Database['Cpu_percent']=cpu_usage_percent[i]
        c_count=psutil.cpu_count()
        temp=cpu_usage_percent[i]
        temp=temp/100
        c_count=c_count*temp
        c_count=round(c_count,2)
        Database['Cpu_usage']=c_count
        Database['Memory_usage']=memmory_usage[i]
        Database['Select_AllData_time']=select_time[i]
        writer.writerow(Database)
'''
selectAllQuery('test','test1')
selectAllQuery('test','test2')
selectAllQuery('test','test3')
selectAllQuery('test','test4')
selectAllQuery('test','test5')
#從string轉為nparray

#print(data[0]['data'])
#import numpy as np
#d = np.fromstring(data[0]['data'][1:-1], sep=' ')
#print(d.shape)
'''

Case:  1   20萬筆資料
insertTest
Time: 0:00:00.044724
selectAllQuery
Time: 0:00:00.017473
selectStationQuery
Time: 0:00:00.004625
selectDateRangeQuery
Time: 0:00:00.004062
selectDateRangeCountQuery
Time: 0:00:00.001713
pipelineQuery
Time: 0:00:00.004888
CPU used in 0 seconds: 15 Percent
Memory used in 0 seconds: 2.67 GB
Case:  2   5萬筆資料
insertTest
Time: 0:00:00.024317
selectAllQuery
Time: 0:00:00.010551
selectStationQuery
Time: 0:00:00.004678
selectDateRangeQuery
Time: 0:00:00.004294
selectDateRangeCountQuery
Time: 0:00:00.001415
pipelineQuery
Time: 0:00:00.004833
CPU used in 0 seconds: 28 Percent
Memory used in 0 seconds: 2.67 GB


"\nselectAllQuery('test','test1')\nselectAllQuery('test','test2')\nselectAllQuery('test','test3')\nselectAllQuery('test','test4')\nselectAllQuery('test','test5')\n#從string轉為nparray\n\n#print(data[0]['data'])\n#import numpy as np\n#d = np.fromstring(data[0]['data'][1:-1], sep=' ')\n#print(d.shape)\n"

In [8]:
print(cpu1)

[27.8, 0.0, 0.0, 50.0, 0.0, 0.0, 0.0, 100.0, 0.0, 0.0]
